<a href="https://colab.research.google.com/github/KrishK1996/DO101-apps/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

# Path to your Excel file in Drive
excel_path = '/content/drive/MyDrive/NLP/TBO-main/TBO/TBO_4k_train.xlsx'

# Load the Excel file (if multiple sheets, specify sheet_name)
df = pd.read_excel(excel_path, sheet_name=0)  # 0 = first sheet

# Preview the data
print(df.head())

                    id                                               text  \
0  1158315949990252544  @USER Love it keep up the good work sunshine b...   
1  1161787110359273472  Lmaooo y’all got other women telling y’all tha...   
2  1161808970727743488  @USER I feel that sis. They needa begin deacti...   
3  1186683819274801152  all "rivalry's" and shit talking aside im so h...   
4  1187391554757038081  Black musicians think they have to say a bunch...   

         T1 Target    T1 Argument T1 Harmful        T2 Target   T2 Argument  \
0            @USER    fuck em all         NO              NaN           NaN   
1            y'all  sounds stupid        YES              NaN           NaN   
2           niggas         niggas        YES           niggas  deactivating   
3              NaN   shit talking         NO              NaN           NaN   
4  Black musicians      homo shit        YES  Black musicians           sad   

  T2 Harmful T3 Target T3 Argument T3 Harmful T4 Target T4 Arg

In [9]:
import pandas as pd

# Path to your Excel file
excel_path = '/content/drive/MyDrive/NLP/TBO-main/TBO/TBO_4k_train.xlsx'

# Load Excel
df = pd.read_excel(excel_path, sheet_name=0)

# Display first few rows
print(df.head())


                    id                                               text  \
0  1158315949990252544  @USER Love it keep up the good work sunshine b...   
1  1161787110359273472  Lmaooo y’all got other women telling y’all tha...   
2  1161808970727743488  @USER I feel that sis. They needa begin deacti...   
3  1186683819274801152  all "rivalry's" and shit talking aside im so h...   
4  1187391554757038081  Black musicians think they have to say a bunch...   

         T1 Target    T1 Argument T1 Harmful        T2 Target   T2 Argument  \
0            @USER    fuck em all         NO              NaN           NaN   
1            y'all  sounds stupid        YES              NaN           NaN   
2           niggas         niggas        YES           niggas  deactivating   
3              NaN   shit talking         NO              NaN           NaN   
4  Black musicians      homo shit        YES  Black musicians           sad   

  T2 Harmful T3 Target T3 Argument T3 Harmful T4 Target T4 Arg

In [10]:
# List of harmful columns
harmful_cols = ['T1 Harmful', 'T2 Harmful', 'T3 Harmful', 'T4 Harmful', 'T5 Harmful', 'T6 Harmful']

# Fill NaN with 'NO'
df[harmful_cols] = df[harmful_cols].fillna('NO')

# Create binary label: 1 if any Harmful = YES
df['label'] = df[harmful_cols].apply(lambda x: 1 if 'YES' in x.values else 0, axis=1)

# Keep only 'text' and 'label'
df = df[['text', 'label']]

# Preview
print(df.head())
print(df['label'].value_counts())


                                                text  label
0  @USER Love it keep up the good work sunshine b...      0
1  Lmaooo y’all got other women telling y’all tha...      1
2  @USER I feel that sis. They needa begin deacti...      1
3  all "rivalry's" and shit talking aside im so h...      0
4  Black musicians think they have to say a bunch...      1
label
1    2890
0    1110
Name: count, dtype: int64


In [11]:
import re

def clean_text(text):
    # Remove usernames
    text = re.sub(r'@\w+', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['text'] = df['text'].apply(clean_text)


In [12]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
import torch

# Load pre-trained base model (just for demonstration)
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

# Put model in eval mode
model.eval()


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [13]:
def predict_offensive(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        pred = torch.argmax(outputs.logits, dim=-1).item()
    return "Offensive" if pred == 1 else "Non-offensive"


In [14]:
examples = [
    "I hate you, you idiot!",
    "You are so smart and kind!",
    "This is ridiculous!"
]

for text in examples:
    label = predict_offensive(text)
    print(f"Text: {text}\nPredicted Label: {label}\n")


Text: I hate you, you idiot!
Predicted Label: Offensive

Text: You are so smart and kind!
Predicted Label: Offensive

Text: This is ridiculous!
Predicted Label: Offensive



In [ ]:
while True:
    text = input("Enter a sentence (or 'exit' to quit): ")
    if text.lower() == 'exit':
        break
    print("Prediction:", predict_offensive(text))


Enter a sentence (or 'exit' to quit): You are fool and idiot stupic
Prediction: Offensive
Enter a sentence (or 'exit' to quit): duffer
Prediction: Offensive
Enter a sentence (or 'exit' to quit): cancer fellow
Prediction: Offensive
Enter a sentence (or 'exit' to quit): Waste fellow
Prediction: Offensive
